In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

full = pd.read_csv("../../data/poptrag_lyrics_genres_corpus_filtered_english_lemmatized.csv")
full.info()

# top 20 most common words in the lyrics
# def print_most_common_words(corpus, lyrics_column, top_n=20):
#     vectorizer = CountVectorizer(
#                 ngram_range=(1, 1),
#                 token_pattern=r"\b[\w']+\b",
#                 lowercase=True,
#             )
#     matrix = vectorizer.fit_transform(full[lyrics_column])
#     sum_words = matrix.sum(axis=0)
#     words_freq = [(word, sum_words[0, idx]) for word, idx in vectorizer.vocabulary_.items()]
#     words_freq = sorted(words_freq, key=lambda x: x[1], reverse=True)
#     print(words_freq[:20])

# print_most_common_words(full, "full_lyrics")
# print("=" * 60)
# print_most_common_words(full, "lyrics_lemmatized")

<class 'pandas.DataFrame'>
RangeIndex: 111938 entries, 0 to 111937
Data columns (total 22 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   Unnamed: 0                111938 non-null  int64  
 1   track.s.id                111938 non-null  str    
 2   track.s.title             111937 non-null  str    
 3   track.s.firstartist.name  111938 non-null  str    
 4   album.s.title             111938 non-null  str    
 5   album.s.releaseyear       111938 non-null  int64  
 6   track.s.popularity        111938 non-null  int64  
 7   track.language            111938 non-null  str    
 8   full_lyrics               111938 non-null  str    
 9   cat5                      111938 non-null  str    
 10  pmax5                     111938 non-null  float64
 11  nmax5                     111938 non-null  float64
 12  cat12                     111938 non-null  str    
 13  pmax12                    111938 non-null  float64
 14 

In [6]:
from helpers.LyricsClassficationExperiment import LyricsClassificationExperiment

exp_fs = LyricsClassificationExperiment(
    corpus=full, 
    genrecol="cat12",
    lyricscol="full_lyrics", 
    artistcol="track.s.firstartist.name", 
    output_dir="cat5_mock_experiment_fs",
    test_size=0.2,
    random_state=42, 
    subsample_debug=0.01,
)
exp_fs.compute_fs_ngram_features(min_artists=20, top_n=100)
fs_features = exp_fs.X_train.keys()
print(exp_fs)
print(type(exp_fs.X_train))
exp_fs.train_fixed_parametrer_logistic_regression()
exp_fs.show_model_evaluation()
exp_fs.show_top_coefficients_per_genre()

✓ Extracted unigrams:
  - Unique: 9,057
  - Shape: (893, 9057)
  - Examples: ['cuffs', 'asphyxiated', 'lies', 'inspire', 'hersheys']
✓ Extracted bigrams:
  - Unique: 55,891
  - Shape: (893, 55891)
  - Examples: ['stay or', 'bueno chinga', 'aimlessly but', 'town yeah', 'goodbye the']
✓ Extracted trigrams:
  - Unique: 95,312
  - Shape: (893, 95312)
  - Examples: ['wait in silence', 'challenged all across', 'all made the', "i'm off st", 'hit oh by']
Calculating genre-level TF-IDF for unigrams with genre ...
✓ Calculated TF-IDF for 19,739 genre-ngram pairs
Calculating genre-level TF-IDF for bigrams with genre ...
✓ Calculated TF-IDF for 74,810 genre-ngram pairs
Calculating genre-level TF-IDF for trigrams with genre ...
✓ Calculated TF-IDF for 101,923 genre-ngram pairs
Counting artists per n-gram...
  10% complete
  20% complete
  30% complete
  40% complete
  50% complete
  60% complete
  70% complete
  80% complete
  90% complete
✓ Calculated artist diversity for 9,057 n-grams
Counting ar

N:\Materialien\Promotion\LyricsGenreRecognition\venv\Lib\site-packages\sklearn\linear_model\_logistic.py:1135: FutureWarning: 'penalty' was deprecated in version 1.8 and will be removed in 1.10. To avoid this warning, leave 'penalty' set to its default value and use 'l1_ratio' or 'C' instead. Use l1_ratio=0 instead of penalty='l2', l1_ratio=1 instead of penalty='l1', and C=np.inf instead of penalty=None.
  warnings.warn(


Selected model parameters:
  C: 1.000
  l1_ratio: 0.500
  target_ratio: 3.000
F1 macro: 0.169
Precision macro: 0.169
Recall macro: 0.169
Cohen's kappa: 0.130
                  precision    recall  f1-score   support

alternative rock       0.17      0.08      0.11        13
      electronic       0.05      0.05      0.05        19
       hard rock       0.00      0.00      0.00         7
     heavy metal       0.14      0.22      0.17         9
         hip hop       0.43      0.40      0.41        15
      indie rock       0.00      0.00      0.00         5
            jazz       0.00      0.00      0.00         3
           metal       0.33      0.42      0.37        24
             pop       0.35      0.42      0.39        52
        pop rock       0.00      0.00      0.00         2
            rock       0.39      0.34      0.36        77

        accuracy                           0.30       226
       macro avg       0.17      0.18      0.17       226
    weighted avg       0.30 

N:\Materialien\Promotion\LyricsGenreRecognition\venv\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
N:\Materialien\Promotion\LyricsGenreRecognition\venv\Lib\site-packages\sklearn\metrics\_classification.py:1833: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
N:\Materialien\Promotion\LyricsGenreRecognition\venv\Lib\site-packages\sklearn\metrics\_classification.py:1833: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
N:\Materialien\Promotion\LyricsGenreRecognition\venv\Lib\site-packages\sklearn\metrics\_cl

In [4]:
from helpers.LyricsClassficationExperiment import LyricsClassificationExperiment

exp_informed = LyricsClassificationExperiment(
    corpus=full,
    genrecol="cat12",
    lyricscol="lyrics_lemmatized",
    artistcol="track.s.firstartist.name",
    output_dir="cat5_mock_experiment_informed",
    test_size=0.2,
    random_state=42,
    subsample_debug=0.01,
    # subsample_debug=0.1,
)
exp_informed.compute_informed_ngram_features(min_artists=20, top_n=100)
fs_features = exp_informed.X_train.keys()
print(exp_informed)
print(type(exp_informed.X_train))
exp_informed.train_fixed_parametrer_logistic_regression()
exp_informed.show_model_evaluation()

✓ Extracted unigrams:
  - Unique: 7,344
  - Shape: (893, 7344)
  - Examples: ['review', "c'mon", 'almost', 'stole', 'fail']
✓ Extracted bigrams:
  - Unique: 38,109
  - Shape: (893, 38109)
  - Examples: ['cursed beauty', 'alone most', 'look around', "it's one", 'i sure']
✓ Extracted trigrams:
  - Unique: 59,049
  - Shape: (893, 59049)
  - Examples: ['spain to tijuana', 'billy wake up', 'a wild flame', 'those nice thing', 'guess you know']
Boundary stripping bigrams: kept 34,556 / 38,109 (dropped 3,553)
Boundary stripping trigrams: kept 51,282 / 59,049 (dropped 7,767)
Counting artists per n-gram...
  10% complete
  20% complete
  30% complete
  40% complete
  50% complete
  60% complete
  70% complete
  80% complete
  90% complete
✓ Calculated artist diversity for 7,344 n-grams
Counting artists per n-gram...
  10% complete
  20% complete
  30% complete
  40% complete
  50% complete
  60% complete
  70% complete
  80% complete
  90% complete
✓ Calculated artist diversity for 34,556 n-gram

N:\Materialien\Promotion\LyricsGenreRecognition\venv\Lib\site-packages\sklearn\linear_model\_logistic.py:1135: FutureWarning: 'penalty' was deprecated in version 1.8 and will be removed in 1.10. To avoid this warning, leave 'penalty' set to its default value and use 'l1_ratio' or 'C' instead. Use l1_ratio=0 instead of penalty='l2', l1_ratio=1 instead of penalty='l1', and C=np.inf instead of penalty=None.
  warnings.warn(


Selected model parameters:
  C: 1.000
  l1_ratio: 0.500
  target_ratio: 3.000
F1 macro: 0.196
Precision macro: 0.202
Recall macro: 0.202
Cohen's kappa: 0.184
                  precision    recall  f1-score   support

alternative rock       0.00      0.00      0.00        13
      electronic       0.15      0.26      0.19        19
       hard rock       0.20      0.14      0.17         7
     heavy metal       0.12      0.11      0.12         9
         hip hop       0.54      0.47      0.50        15
      indie rock       0.00      0.00      0.00         5
            jazz       0.00      0.00      0.00         3
           metal       0.29      0.29      0.29        24
             pop       0.45      0.48      0.46        52
        pop rock       0.00      0.00      0.00         2
            rock       0.47      0.39      0.43        77

        accuracy                           0.34       226
       macro avg       0.20      0.20      0.20       226
    weighted avg       0.35 

N:\Materialien\Promotion\LyricsGenreRecognition\venv\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [5]:
# print("FS\n" + "=" * 60)
# exp_fs.show_top_coefficients_per_genre()
print("INFORMED\n" + "=" * 60)
exp_informed.show_top_coefficients_per_genre(top_n=20)

INFORMED
Top 20 coefficients for genre: ALTERNATIVE ROCK
oh (0.714)
around (-0.701)
say (-0.649)
heart (0.612)
man (0.576)
somebody (0.569)
see (-0.557)
miss (0.552)
come (-0.543)
like (0.539)
let (0.526)
one (0.509)
old (0.501)
every (0.465)
star (0.453)
lose (0.451)
get (0.432)
eye (-0.406)
fall (0.402)
wonder (0.373)


Top 20 coefficients for genre: ELECTRONIC
end (-0.892)
home (-0.716)
meet (-0.692)
everything (-0.692)
far (-0.641)
blind (-0.637)
sound (-0.625)
heart (-0.616)
flame (0.590)
side (-0.588)
darkness (0.576)
move (-0.568)
always (-0.566)
even (-0.557)
line (-0.525)
goin (-0.522)
door (0.502)
alive (0.501)
break (0.501)
talk (-0.493)


Top 20 coefficients for genre: HARD ROCK
talk (0.720)
save (0.673)
street (0.663)
people (0.560)
wall (0.557)
tonight (0.508)
leave (0.498)
today (0.445)
way (0.435)
last (0.422)
bleed (0.419)
meet (0.411)
pass (0.383)
home (0.369)
stick (0.337)
wild (0.334)
head (0.323)
line (0.318)
behind (-0.317)
reason (0.316)


Top 20 coefficients for

In [6]:
for feautre in exp_informed.X_train.columns:
    print(feautre)



act
ah
ain't
alive
alone
along
always
always be
another
apart
around
ask
away
babe
baby
back
back to
bad
be go
be right
bear
become
bed
begin
behind
believe
big
bitch
black
blame
bleed
blind
blood
blow
blue
body
bout
boy
break
breathe
bring
broken
brother
burn
call
call me
can see
car
care
carry
catch
cause
cause i
cause you
chance
change
child
close
cold
come
come on
come to
control
cry
dark
darkness
day
dead
death
deep
die
don't know
don't need
don't wanna
don't want
door
dream
drink
drop
easy
em
end
enough
even
ever
every
every day
everything
eye
face
faith
fall
far
fear
feel
feel like
feeling
fight
find
fire
flame
fly
forever
forget
free
friend
fuck
full
full of
fun
future
game
get
get it
get me
get my
get to
girl
give
give me
give you
go
go on
go out
go to
god
goin
gonna
good
goodbye
gotta
great
ground
grow
guess
hand
hard
hard to
hate
head
hear
heart
heart be
heaven
hell
hey
hide
high
hit
hold
home
hope
hot
house
hurt
i ain't
i don't know
i feel
i get
i go
i hear
i know
i love
i 

In [9]:
for feautre in exp_fs.X_train.columns:
    print(feautre)

NameError: name 'exp_fs' is not defined